In [1]:
import torch
print(torch.__version__)               # To check PyTorch version
print(torch.cuda.is_available())       # To verify if CUDA is available
print(torch.cuda.get_device_name(0))   # To print the name of your GPU
from torch.utils.data import TensorDataset,DataLoader
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from torch.nn import Sequential,Linear,ReLU,CrossEntropyLoss
import torch.nn.functional as F

2.4.1+cu118
True
NVIDIA GeForce GTX 1650


### Solving Wine Quality classification using simple nn, with three hidden layers, adam optimizer and cross entropy loss function

#### Prepare dataset

In [2]:
wine_data= load_wine()
X,y = wine_data.data,wine_data.target

# train-test-split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)



X_train = torch.tensor(X_train,dtype=torch.float32)
X_test = torch.tensor(X_test,dtype=torch.float32)
y_train = torch.tensor(y_train,dtype=torch.long)
y_test = torch.tensor(y_test,dtype=torch.long)

train_data = TensorDataset(X_train,y_train)
test_data  = TensorDataset(X_test,y_test)

train_data_loader = DataLoader(train_data,batch_size=32,shuffle=True)
test_data_loader = DataLoader(test_data,batch_size=32,shuffle=True)


#### Creating model

In [3]:
model = Sequential(
    Linear(13,64),
    ReLU(),
    Linear(64,32),
    ReLU(),
    Linear(32,16),
    ReLU(),
    Linear(16,3)

)

loss_function = CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
num_epochs = 100

#### Training the model

In [4]:
def fit(num_epochs, model, loss_function, optimizer, data):

    for epoch in range(num_epochs):
        model.train() # this is to set the model to training mode
        running_loss = 0.0
        for input,label in data:
            optimizer.zero_grad() # clear gradients before computations

            output = model(input) # forward pass

            loss = loss_function(output,label) # compute loss
            loss.backward() # backward pass (backpropagation, computing gradients of losses wrt weights and biases, each)
            optimizer.step() # update weights and biases

            running_loss += loss.item()
            
        if (epoch+1)%10 == 0: 
            print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(data):.4f}")




In [5]:
fit(num_epochs,model,loss_function,optimizer,train_data_loader)

Epoch [10/100], Loss: 0.8297
Epoch [20/100], Loss: 0.6965
Epoch [30/100], Loss: 0.6732
Epoch [40/100], Loss: 0.6264
Epoch [50/100], Loss: 0.5892
Epoch [60/100], Loss: 0.5343
Epoch [70/100], Loss: 0.5656
Epoch [80/100], Loss: 0.4880
Epoch [90/100], Loss: 0.3979
Epoch [100/100], Loss: 0.3292


#### Testing the model

In [6]:
def evaluate(model,data):
    model.eval()  # set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for input,label in data:
            output = model(input)
            _, predicted = torch.max(output.data, 1)
            total += label.size(0)
            correct +=(predicted == label).sum().item()
    accuracy  = 100 *correct/total
    return accuracy

evaluate(model,test_data_loader)

88.88888888888889